In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib_inline.backend_inline import set_matplotlib_formats
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import calinski_harabasz_score
from typing import Dict, Optional, Tuple, Any, List
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
from sklearn.metrics import precision_score
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
from scipy import stats
from collections import defaultdict
from math import log2
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import spearmanr
import re, gc
from typing import Optional, Iterable
from time import perf_counter
from datetime import datetime, timedelta

def _rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def _spearman(y_true, y_pred):
    c = spearmanr(y_true, y_pred).correlation
    return float(c) if np.isfinite(c) else np.nan

import warnings
warnings.filterwarnings('ignore')



In [3]:
# 한글 폰트
plt.rc("font", family = 'AppleGothic')
plt.rc("axes", unicode_minus = False)

# 글씨 선명하게
set_matplotlib_formats("retina")

In [4]:
ads_sch = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/스케줄.csv')
print(f"데이터 수 : {ads_sch.shape}")
ads_sch.head()

데이터 수 : (527696, 11)


,sch_idx,ads_idx,sch_type,mda_idx_arr,sch_sdatetime,sch_edatetime,sch_adv_pay,sch_ads_pay,sch_mda_pay,sch_clk_num,sch_turn_num
0,1257478,463989,M,"562,563",2025-09-11 09:42:03,9999-12-31 23:59:59,18,18,14,99999999,14
1,1257477,463989,A,NaN,2025-09-11 09:42:03,9999-12-31 23:59:59,18,18,12,99999999,14
2,1257476,463988,M,"562,563",2025-09-11 09:42:02,9999-12-31 23:59:59,18,18,14,99999999,18
3,1257475,463988,A,NaN,2025-09-11 09:42:02,9999-12-31 23:59:59,18,18,12,99999999,18
4,1257474,463987,M,"562,563",2025-09-11 09:41:55,9999-12-31 23:59:59,18,18,14,99999999,7


In [5]:
ads_sch.columns

Index(['sch_idx', 'ads_idx', 'sch_type', 'mda_idx_arr', 'sch_sdatetime',
       'sch_edatetime', 'sch_adv_pay', 'sch_ads_pay', 'sch_mda_pay',
       'sch_clk_num', 'sch_turn_num'],
      dtype='object')

In [6]:
ads_time = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정_시간별적립보고서(최종).csv')
print(f"데이터 수 : {ads_time.shape}")
ads_time.head()

데이터 수 : (279916, 10)


,rpt_time_date,rpt_time_time,ads_idx,mda_idx,rpt_time_clk,rpt_time_turn,rpt_time_acost,rpt_time_earn,weekday,week_type
0,2025-07-27,0,11059,398,5,0,0,0,6,주말
1,2025-07-27,0,13209,583,3,0,0,0,6,주말
2,2025-07-27,0,13928,303,3,3,510,360,6,주말
3,2025-07-27,0,13928,324,3,3,510,360,6,주말
4,2025-07-27,0,13928,340,6,6,1020,720,6,주말


In [7]:
ads_time.columns

Index(['rpt_time_date', 'rpt_time_time', 'ads_idx', 'mda_idx', 'rpt_time_clk',
       'rpt_time_turn', 'rpt_time_acost', 'rpt_time_earn', 'weekday',
       'week_type'],
      dtype='object')

In [8]:
ads_domain = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/광고도메인리스트_1년치.csv')
print(f"데이터 수 : {ads_domain.shape}")
ads_domain.head()

데이터 수 : (233408, 24)


,Unnamed: 0,domain,row_id,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,...,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate
0,0,금융,153088,154946,KoM69LPFUB,8,50,3,7,어센틱금융그룹 내보험조회,...,2025-02-12 10:00:00,2029-11-29 00:00:00,0,100,7,800,640,2444400,NONE,2025-02-12 10:18:46
1,21,게임,135518,137375,RBjgpLVOmR,71,910,3,8,DK모바일 리본 사전예약,...,2025-01-10 18:00:00,2030-01-01 00:00:00,0,100,7,200,150,2431500,NONE,2025-01-10 18:48:52
2,23,게임,427141,429013,TUSVjcBYtK,76,1355,3,5,인생존망겜(레벨 67 달성),...,2025-07-11 16:23:32,9999-12-31 23:59:59,0,100,2,9600,7200,2460500,NONE,2025-07-11 16:23:32
3,31,게임,161321,163182,OLNpNxxGJm,76,1355,3,5,고블린 우드: 타이쿤 유휴 심,...,2025-02-27 16:00:00,9999-12-31 23:59:59,0,100,2,4480,3360,2451800,NONE,2025-02-26 17:05:49
4,34,게임,161392,163254,r78MSmvuQk,76,1355,3,5,Save the Pets,...,2025-02-27 16:00:00,2031-01-03 00:00:00,0,100,1,1440,1080,2450800,NONE,2025-02-26 17:57:33


In [9]:
ads_domain.columns

Index(['Unnamed: 0', 'domain', 'row_id', 'ads_idx', 'ads_code', 'aff_idx',
       'adv_idx', 'ads_type', 'ads_category', 'ads_name', 'ads_icon_img',
       'ads_summary', 'ads_save_way', 'ads_day_cap', 'ads_sdate', 'ads_edate',
       'ads_age_min', 'ads_age_max', 'ads_os_type', 'ads_contract_price',
       'ads_reward_price', 'ads_order', 'ads_rejoin_type', 'regdate'],
      dtype='object')

In [10]:
ads_list = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정2_광고목록.csv')
print(f"데이터 수 : {ads_list.shape}")
ads_list.head()

데이터 수 : (445039, 21)


,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,...,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate
0,160,tr71opm6C1,6,10,1,1,리니지레드나이츠,https://lh3.googleusercontent.com/oDGXxngO9oOP...,[앱설명]<br />[설치형] 매력만점 영웅들과 소환수들이 함께하는 취향저격 RPG...,받기,...,2016-12-01 00:00:00,9999-12-31 23:59:59,0,100,2,170,150,982800,NONE,2017-04-06 13:19:53
1,284,xlNpfkgtLX,6,10,1,1,강철의함대:Ocean Overlord,http://nextapps-nas.aws.appang.kr/icon/2016122...,"[앱설명]<br />[설치형] 한, 중, 일 아시아 유저들과 함께 대규모 해상 전투...",받기,...,2016-12-29 00:00:00,9999-12-31 23:59:59,0,100,2,180,160,987200,NONE,2017-01-10 10:37:22
2,292,7Hj8V5QKiy,6,10,1,1,스노우 SNOW,http://nextapps-nas.aws.appang.kr/icon/2016092...,"[앱설명]<br />[설치형] 셀카, 얼굴인식 스티커, 꿀잼 카메라<br /><br...",받기,...,2016-12-29 00:00:00,9999-12-31 23:59:59,0,100,2,160,160,987300,NONE,2017-01-02 12:51:34
3,304,1RfxHJcuEe,6,10,2,1,서머너즈 워: 천공의 아레나,http://nextapps-nas.aws.appang.kr/icon/EecpHzP...,[앱설명]<br />[실행형] 전 세계 5000만 소환사들을 사로잡은 명작 RPG<...,최초 오픈,...,2017-01-01 00:00:00,9999-12-31 23:59:59,0,100,2,230,210,931000,NONE,2017-03-30 17:05:23
4,306,l3GyilYEVk,6,10,1,1,하이마트,http://nextapps-nas.aws.appang.kr/icon/2016032...,새롭게 단장한 하이마트 쇼핑몰 앱!,받기,...,2017-01-01 00:00:00,2019-11-01 00:00:00,0,100,2,170,140,1155100,NONE,2017-06-12 15:24:46


In [11]:
ads_list.columns

Index(['ads_idx', 'ads_code', 'aff_idx', 'adv_idx', 'ads_type', 'ads_category',
       'ads_name', 'ads_icon_img', 'ads_summary', 'ads_save_way',
       'ads_day_cap', 'ads_sdate', 'ads_edate', 'ads_age_min', 'ads_age_max',
       'ads_os_type', 'ads_contract_price', 'ads_reward_price', 'ads_order',
       'ads_rejoin_type', 'regdate'],
      dtype='object')

In [12]:
ads_rwd = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정_광고적립.csv')
print(f"데이터 수 : {ads_rwd.shape}")
ads_rwd.head()

데이터 수 : (1475031, 14)


,rwd_idx,ads_idx,ads_code,mda_idx,pub_sub_rel_id,dvc_idx,advid,click_key,adv_cost,earn_cost,click_date,regdate,ctit,cost_diff
0,238304818,413490,CFZ6lzEmeD,22,27496512,32947806,09b6a808-4f76-4584-87c5-626f21f472a0,b0e22dba943810794d25c993dff425c4e16d0744,190,120,2025-07-26 00:00:31,2025-07-26 00:05:02,271,70
1,238304829,413490,CFZ6lzEmeD,22,27496512,47387244,ba4fd0c8-4f06-4dfa-9c3f-c6b39ee7066e,0679ff4c6a14beba296e44d7ffdd88129f42ffbf,190,120,2025-07-26 00:00:32,2025-07-26 00:08:17,465,70
2,238304816,413490,CFZ6lzEmeD,22,27496512,32425124,64c675d0-8edd-4478-9906-a806651fdf92,68eb28d636724061da21e490a1afff6a39cd78f7,190,120,2025-07-26 00:00:42,2025-07-26 00:04:43,241,70
3,238304817,413490,CFZ6lzEmeD,22,27496513,34006545,a08ebb04-e70a-4492-89d4-17fa96bf8c75,37da42f12b6f732d8c8bf1a609c01e569b7f8a09,190,120,2025-07-26 00:00:54,2025-07-26 00:04:45,231,70
4,238304802,133653,rl8ud4uNQC,801,1,56867331,dbe54f40-41b5-4423-ae2f-048fd6ecf5af,624b1bbc57c9ea81005c74e1956d48e76d974816,260,195,2025-07-26 00:01:07,2025-07-26 00:02:04,57,65


In [13]:
ads_rwd.columns

Index(['rwd_idx', 'ads_idx', 'ads_code', 'mda_idx', 'pub_sub_rel_id',
       'dvc_idx', 'advid', 'click_key', 'adv_cost', 'earn_cost', 'click_date',
       'regdate', 'ctit', 'cost_diff'],
      dtype='object')

In [14]:
ads_info = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정2_광고참여정보.csv')
print(f"데이터 수 : {ads_info.shape}")
ads_info.head()

데이터 수 : (16854865, 14)


,click_key,ads_idx,dvc_idx,mda_idx,pub_sub_rel_id,contract_price,media_price,click_day,click_time,click_date,exp_day,network,user_ip,converted
0,000000d54b9faad47ee99d6cd3cf53894dd4baa5,313780,61906528,539,144350110,6000,4500,2025-08-17,21,2025-08-17 21:07:37,2025-09-16,0,16.184.28.219,0
1,000002b4d92f7648b455877c2676452efcd22a09,412426,34422806,58,46032732,180,170,2025-07-26,2,2025-07-26 02:18:24,2025-08-25,NaN,35.78.117.76,1
2,0000057e97361ff3d0263aaecee34cfaa3ba30fb,443660,38366075,808,1,170,120,2025-08-12,18,2025-08-12 18:17:59,2025-09-11,WIFI,39.7.55.192,1
3,00000607f60139015da3ee1dd5499db3faa100dc,360192,61894110,539,144350110,6000,4500,2025-08-17,3,2025-08-17 03:35:02,2025-09-16,0,211.252.100.157,0
4,0000066bc25d4a6d147c27326cf972a4de88024e,372307,61956954,539,144350110,15600,11700,2025-08-18,8,2025-08-18 08:01:26,2025-09-17,0,3.38.148.211,0


In [15]:
ads_info.columns

Index(['click_key', 'ads_idx', 'dvc_idx', 'mda_idx', 'pub_sub_rel_id',
       'contract_price', 'media_price', 'click_day', 'click_time',
       'click_date', 'exp_day', 'network', 'user_ip', 'converted'],
      dtype='object')

In [16]:
ads_join = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/참여적립조인.csv')
print(f"데이터 수 : {ads_join.shape}")
ads_join.head()

데이터 수 : (16854865, 22)


,Unnamed: 0,click_key,ads_idx,dvc_idx,mda_idx,pub_sub_rel_id,contract_price,media_price,click_day,click_time,...,network,user_ip,rwd_idx,ads_code,advid,adv_cost,earn_cost,regdate,ctit,conversion
0,0,000000d54b9faad47ee99d6cd3cf53894dd4baa5,313780,61906528,539,144350110,6000,4500,2025-08-17,21,...,0,16.184.28.219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,000002b4d92f7648b455877c2676452efcd22a09,412426,34422806,58,46032732,180,170,2025-07-26,2,...,NaN,35.78.117.76,238313226.0,yuRe4YoPuZ,40ceb01c-9b60-4452-bacd-0e061fb18ab4,180.0,170.0,2025-07-26 02:19:21,57.0,1
2,2,0000057e97361ff3d0263aaecee34cfaa3ba30fb,443660,38366075,808,1,170,120,2025-08-12,18,...,WIFI,39.7.55.192,239017177.0,WuIp8HKmt9,73793f44-a471-4351-ae1c-d09accaf223a,170.0,120.0,2025-08-12 18:19:21,82.0,1
3,3,00000607f60139015da3ee1dd5499db3faa100dc,360192,61894110,539,144350110,6000,4500,2025-08-17,3,...,0,211.252.100.157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4,0000066bc25d4a6d147c27326cf972a4de88024e,372307,61956954,539,144350110,15600,11700,2025-08-18,8,...,0,3.38.148.211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [17]:
ads_join.columns

Index(['Unnamed: 0', 'click_key', 'ads_idx', 'dvc_idx', 'mda_idx',
       'pub_sub_rel_id', 'contract_price', 'media_price', 'click_day',
       'click_time', 'click_date', 'exp_day', 'network', 'user_ip', 'rwd_idx',
       'ads_code', 'advid', 'adv_cost', 'earn_cost', 'regdate', 'ctit',
       'conversion'],
      dtype='object')

In [18]:
# ads_total = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/아이브1년치_참여데이터_통합.csv')
# print(f"데이터 수 : {ads_total.shape}")
# ads_total.head()

In [19]:
ads_segment = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/ads_segment.csv')
print(f"데이터 수 : {ads_segment.shape}")
ads_segment.head()

데이터 수 : (9517, 40)


,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,...,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster
0,0,6508,1,1,2,0,10,금융,7,2381500,...,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
1,1,6985,2,30,46,0,7,금융,7,1201938,...,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0
2,2,8327,1,6,11,0,10,미디어/컨텐츠,7,1673458,...,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0
3,3,9264,1,1,1,0,10,금융,7,2372800,...,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
4,4,9716,2,4,6,1,8,생활,2,2699900,...,LOW,MEDIUM_LOW,2,0,1,1,0,4,SMALL,0


In [20]:
ads_segment.columns

Index(['Unnamed: 0', 'ads_idx', 'media_count', 'user_count', 'total_clicks',
       'total_conversions', 'ads_category', 'domain', 'ads_os_type',
       'ads_order', 'ctit_mean', 'ctit_median', 'ads_rejoin_type',
       'contract_price', 'media_price', 'first_click', 'last_click',
       'ads_name', 'ads_sdate', 'expire', 'days_active',
       'daily_avg_conversions', 'cvr', 'margin', 'roi', 'total_net_return',
       'ads_level_detailed', 'daily_clicks', 'daily_users', 'scale_level',
       'performance_level', 'ads_level', 'media_score', 'conv_score',
       'clicks_score', 'stability_score', 'cvr_score', 'total_score',
       'ads_size', 'cluster'],
      dtype='object')

In [21]:
media_portfolio = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/media_portfolio.csv')
print(f"데이터 수 : {media_portfolio.shape}")
media_portfolio.head()

데이터 수 : (189, 50)


,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,...,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit
0,0,12,75700,232438,82938,2025-07-26 00:00:02,2025-08-25 11:22:16,31,2675.419355,7,...,16.666667,12.500000,0.00000,16.666667,0.000000,0.000000,4.166667,0.000000,12.500000,3003668
1,1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,31,...,0.000000,13.559322,0.00000,3.389831,0.000000,0.000000,3.389831,1.694915,5.084746,2733547
2,2,18,2825,5961,250,2025-07-26 00:00:49,2025-08-25 11:13:19,31,8.064516,19,...,0.000000,8.695652,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,4.347826,182366
3,3,22,37668,83355,35232,2025-07-26 00:00:31,2025-08-25 11:14:50,31,1136.516129,67,...,3.012048,11.445783,0.60241,4.216867,1.204819,7.831325,4.819277,1.204819,7.228916,2541770
4,4,26,402,678,70,2025-07-26 01:45:31,2025-08-25 10:22:36,31,2.258065,10,...,0.000000,10.526316,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,10.526316,2690


In [22]:
media_portfolio.columns

Index(['Unnamed: 0', 'mda_idx', 'user_count', 'total_clicks',
       'total_conversions', 'first_click', 'last_click', 'days_active',
       'daily_avg_conversions', 'LARGE', 'MEDIUM', 'MEGA', 'SMALL',
       'total_ads', 'MEGA_ratio', 'LARGE_ratio', 'MEDIUM_ratio', 'SMALL_ratio',
       'ads_category_0_pct', 'ads_category_1_pct', 'ads_category_2_pct',
       'ads_category_3_pct', 'ads_category_4_pct', 'ads_category_5_pct',
       'ads_category_6_pct', 'ads_category_7_pct', 'ads_category_8_pct',
       'ads_category_10_pct', 'ads_category_11_pct', 'ads_category_13_pct',
       'ads_os_type_1_pct', 'ads_os_type_2_pct', 'ads_os_type_3_pct',
       'ads_os_type_7_pct', 'domain_게임_pct', 'domain_교육_pct', 'domain_금융_pct',
       'domain_기타_pct', 'domain_미디어/컨텐츠_pct', 'domain_뷰티_pct',
       'domain_비영리/공공_pct', 'domain_생활_pct', 'domain_식당/카페_pct',
       'domain_식음료_pct', 'domain_운동/스포츠_pct', 'domain_운세_pct',
       'domain_의료/건강_pct', 'domain_채용_pct', 'domain_커머스_pct',
       'expected_tota

In [23]:
# ==============================
# Ultra-Light Attribute-Only Ranker (Crash-safe)
# ==============================

# ---------- utils ----------
def _to_num(s): 
    return pd.to_numeric(s, errors="coerce").fillna(0)

def _to_dt(s):
    return pd.to_datetime(s, errors="coerce")

def split_obs_tgt_month(ads_time: pd.DataFrame, tgt_days: int = 7):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    last_date = t["date"].max()
    month_start = last_date.normalize().replace(day=1)
    next_month  = (month_start + pd.offsets.MonthBegin(1))
    month_end   = (next_month - pd.Timedelta(days=1)).normalize()
    cutoff      = month_end - pd.Timedelta(days=tgt_days)
    obs = t[(t["date"] >= month_start) & (t["date"] <= cutoff)].copy()
    tgt = t[(t["date"] >  cutoff) & (t["date"] <= month_end)].copy()
    return obs, tgt, month_start, month_end, cutoff

def sanitize_feature_names(df: pd.DataFrame, feat_cols):
    ren, used = {}, set()
    for c in feat_cols:
        safe = re.sub(r'[^A-Za-z0-9_]', '_', str(c))
        if safe == '' or safe[0].isdigit():
            safe = f"f_{safe}"
        base, k = safe, 1
        while safe in used:
            k += 1; safe = f"{base}_{k}"
        ren[c] = safe; used.add(safe)
    df = df.rename(columns=ren)
    feat_cols_safe = [ren.get(c, c) for c in feat_cols]
    return df, feat_cols_safe, ren

def ensure_feature_matrix(df, feat_cols):
    for c in feat_cols:
        if c not in df.columns:
            df[c] = 0.0
    X = df[feat_cols].copy()
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors="coerce")
    return X.fillna(0.0).astype(np.float32)

# ---------- metrics ----------
def precision_at_k(gt_set, pred_list, k):
    if k <= 0: return np.nan
    k = min(k, len(pred_list))
    return sum(1 for a in pred_list[:k] if a in gt_set) / max(1, k)

def recall_at_k(gt_set, pred_list, k):
    if len(gt_set) == 0: return np.nan
    return sum(1 for a in pred_list[:k] if a in gt_set) / len(gt_set)

def average_precision_at_k(gt_set, pred_list, k):
    if len(gt_set) == 0: return np.nan
    hits, ap = 0, 0.0
    for i, a in enumerate(pred_list[:k], start=1):
        if a in gt_set:
            hits += 1; ap += hits / i
    return ap / min(len(gt_set), k)

def ndcg_at_k_from_gains(gains_in_pred_order, k):
    k = min(k, len(gains_in_pred_order))
    dcg  = sum(g / log2(i+2) for i, g in enumerate(gains_in_pred_order[:k]))
    ideal = sorted(gains_in_pred_order, reverse=True)[:k]
    idcg = sum(g / log2(i+2) for i, g in enumerate(ideal))
    return dcg / idcg if idcg > 0 else 0.0

def ndcg_at_k(gt_gain_map, pred_list, k):
    gains = [gt_gain_map.get(a, 0.0) for a in pred_list[:k]]
    return ndcg_at_k_from_gains(gains, k)

def hit_rate_at_k(gt_set, pred_list, k):
    if len(gt_set) == 0: return np.nan
    return 1.0 if any(a in gt_set for a in pred_list[:k]) else 0.0

def evaluate_rankings(pred_dict, gt_dict, ks=(5,10,20)):
    rows = []
    for ad, preds in pred_dict.items():
        if ad not in gt_dict: 
            continue
        gt_map = gt_dict[ad]; gt_set = set(gt_map.keys())
        if len(preds) == 0: 
            continue
        row = {"ads_idx": ad, "gt_size": len(gt_set)}
        for k in ks:
            row[f"Precision@{k}"] = precision_at_k(gt_set, preds, k)
            row[f"Recall@{k}"]    = recall_at_k(gt_set, preds, k)
            row[f"MAP@{k}"]       = average_precision_at_k(gt_set, preds, k)
            row[f"NDCG@{k}"]      = ndcg_at_k(gt_map, preds, k)
            row[f"HitRate@{k}"]   = hit_rate_at_k(gt_set, preds, k)
        rows.append(row)
    per_ad = pd.DataFrame(rows)
    summary = {"n_ads_eval": int(per_ad.shape[0])}
    for k in ks:
        for m in ["Precision","Recall","MAP","NDCG","HitRate"]:
            col = f"{m}@{k}"
            summary[col] = float(per_ad[col].mean(skipna=True)) if col in per_ad else np.nan
    summary["avg_gt_size"] = float(per_ad["gt_size"].mean(skipna=True)) if "gt_size" in per_ad else np.nan
    return summary, per_ad

# ---------- attribute-only data ----------
def _make_domain_mapping(media_portfolio: pd.DataFrame):
    dom_cols = [c for c in media_portfolio.columns if c.startswith("domain_") and c.endswith("_pct")]
    cat2dom = {
        0:"domain_커머스_pct", 1:"domain_게임_pct", 2:"domain_교육_pct", 3:"domain_금융_pct",
        4:"domain_뷰티_pct",   5:"domain_생활_pct", 6:"domain_식음료_pct", 7:"domain_운동/스포츠_pct",
        8:"domain_운세_pct",   9:"domain_의료/건강_pct", 10:"domain_채용_pct", 11:"domain_미디어/컨텐츠_pct",
        12:"domain_식당/카페_pct", 13:"domain_비영리/공공_pct", 14:"domain_기타_pct",
    }
    cat2dom = {k:v for k,v in cat2dom.items() if v in dom_cols}
    return dom_cols, cat2dom

def _global_media_perf_from_obs(obs):
    if obs is None or len(obs)==0:
        return pd.DataFrame({"mda_idx":[], "glob_turn":[], "glob_clk":[], "glob_ctr":[], "glob_pop_z":[]})
    t = obs.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    perf = (t.groupby("mda_idx", as_index=False)
              .agg(glob_turn=("turn","sum"), glob_clk=("clk","sum")))
    perf["glob_ctr"] = (perf["glob_turn"] / perf["glob_clk"].replace(0,np.nan)).fillna(0.0)
    z = perf["glob_turn"].astype(float)
    perf["glob_pop_z"] = (z - z.mean()) / (z.std() + 1e-9)
    return perf

def build_attribute_pairs_ultra_safe(
    ads_list, media_portfolio, obs=None, tgt=None,
    mda_per_ad=150, top_mda_frac=0.25, ads_limit=None, rng_seed=42
):
    # 광고/매체 기본 테이블
    A = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    if ads_limit is not None and len(A) > ads_limit:
        A = A.sample(n=int(ads_limit), random_state=rng_seed).reset_index(drop=True)
    M = media_portfolio.drop_duplicates("mda_idx").copy()

    # 수치화
    for c in M.columns:
        if c!="mda_idx":
            M[c] = pd.to_numeric(M[c], errors="coerce").fillna(0.0)

    # 전역 매체 성과(허용 범위)
    mperf = _global_media_perf_from_obs(obs)
    M = M.merge(mperf, on="mda_idx", how="left").fillna(0.0)

    # 도메인 매핑
    _, CAT2DOM = _make_domain_mapping(M)

    # 인기 상위 mda 선정
    if "glob_pop_z" in M.columns and len(M)>0:
        M_sorted = M.sort_values("glob_pop_z", ascending=False).reset_index(drop=True)
    else:
        M_sorted = M.sample(frac=1.0, random_state=rng_seed).reset_index(drop=True)

    rng = np.random.RandomState(rng_seed)
    rows = []
    mda_all = M["mda_idx"].to_numpy()
    top_cut = int(max(0, round(mda_per_ad * top_mda_frac)))
    top_ids = M_sorted["mda_idx"].head(max(1, top_cut)).to_numpy()

    for _, a in A.iterrows():
        # 후보 샘플링: 상위 일부 + 랜덤
        rem = max(0, mda_per_ad - len(top_ids))
        if rem > 0:
            rest_pool = np.setdiff1d(mda_all, top_ids, assume_unique=False)
            rnd = rng.choice(rest_pool, size=min(rem, len(rest_pool)), replace=False) if len(rest_pool)>0 else np.array([], dtype=int)
            pick = np.concatenate([top_ids, rnd]) if len(top_ids)>0 else rnd
        else:
            pick = top_ids[:mda_per_ad]
        g = M.loc[M["mda_idx"].isin(pick)].copy()
        g.insert(0, "ads_idx", int(a["ads_idx"]))
        g.insert(1, "ads_category", a["ads_category"])

        # domain 적합도(간단)
        def _dom_val(row):
            col = CAT2DOM.get(int(a["ads_category"])) if pd.notna(a["ads_category"]) else None
            return float(row.get(col, 0.0)) if col in row.index else 0.0
        g["domain_fit_simple"] = g.apply(_dom_val, axis=1)
        rows.append(g)

    pairs = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=["ads_idx","mda_idx"])
    del rows; gc.collect()

    # 라벨(옵션): TGT 전환수
    if tgt is not None and len(tgt):
        tg = (tgt.copy()
                .assign(turn=_to_num(tgt.get("rpt_time_turn", 0)))
                .groupby(["ads_idx","mda_idx"], as_index=False)
                .agg(tgt_turn=("turn","sum")))
        pairs = pairs.merge(tg, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_turn":0.0})
    else:
        pairs["tgt_turn"] = 0.0

    # 피처 목록
    drop_cols = {"tgt_turn"}
    key_cols  = {"ads_idx","mda_idx","ads_category"}
    feat_cols = [c for c in pairs.columns if c not in drop_cols.union(key_cols)]
    return pairs, feat_cols

def make_relevance_balanced(df: pd.DataFrame, n_bins=3):
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum() > 0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=n_bins, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int) + 1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df

# ---------- train / predict ----------
def train_attr_ranker_ultra_safe(df_attr, feat_cols, tgt, valid_ratio=0.2, random_state=42):
    # 정답이 있는 광고만 사용(평가 가능하도록)
    ads_with_gt = set(tgt.loc[_to_num(tgt["rpt_time_turn"])>0, "ads_idx"].unique().tolist())
    df_attr = df_attr[df_attr["ads_idx"].isin(ads_with_gt)].copy()
    if df_attr.empty:
        raise RuntimeError("TGT에 정답이 있는 광고가 없어 평가가 불가합니다.")

    df_attr = make_relevance_balanced(df_attr, n_bins=3)

    ads_all = df_attr["ads_idx"].unique()
    rng     = np.random.RandomState(random_state)
    rng.shuffle(ads_all)
    n_valid = max(1, int(len(ads_all)*valid_ratio))
    valid_ads = set(ads_all[:n_valid])

    tr = df_attr[~df_attr["ads_idx"].isin(valid_ads)].copy()
    va = df_attr[ df_attr["ads_idx"].isin(valid_ads)].copy()

    # sanitize (중요!)
    tr, feat_cols_safe, ren_map = sanitize_feature_names(tr, feat_cols)
    va = va.rename(columns=ren_map)

    X_tr = ensure_feature_matrix(tr, feat_cols_safe)
    X_va = ensure_feature_matrix(va, feat_cols_safe)
    y_tr = tr["relevance"].astype(int)
    y_va = va["relevance"].astype(int)
    group_tr = tr.groupby("ads_idx").size().astype(int).tolist()
    group_va = va.groupby("ads_idx").size().astype(int).tolist()

    # 경량 + 안정화 파라미터
    ranker = lgb.LGBMRanker(
        objective="lambdarank", metric="ndcg", eval_at=[10],
        learning_rate=0.05, n_estimators=300, num_leaves=15,
        min_data_in_leaf=120, lambda_l2=2.0,
        feature_fraction=0.8, subsample=0.9, bagging_freq=1,
        max_bin=63, min_data_in_bin=3,
        force_col_wise=True,          # 메모리/안정성 ↑
        deterministic=True,           # 재현성 ↑
        random_state=random_state
    )
    ranker.fit(
        X_tr, y_tr,
        group=group_tr,
        eval_set=[(X_va, y_va)],
        eval_group=[group_va],
        callbacks=[lgb.early_stopping(80), lgb.log_evaluation(50)]
    )
    print("best_iteration_", getattr(ranker, "best_iteration_", None))
    return ranker, valid_ads, feat_cols_safe, ren_map

def attr_predict_for_ad_ultra_safe(ranker, feat_cols_safe, ren_map, ads_idx, media_portfolio, ads_list,
                                   mda_per_ad=150, top_mda_frac=0.25):
    # 광고 1개 그리드 (샘플링 방식 동일)
    a = ads_list.loc[ads_list["ads_idx"]==ads_idx, ["ads_idx","ads_category"]].drop_duplicates("ads_idx")
    if a.empty: raise ValueError("알 수 없는 ads_idx")
    # 매체/전역 성과
    M  = media_portfolio.drop_duplicates("mda_idx").copy()
    for c in M.columns:
        if c!="mda_idx": M[c] = pd.to_numeric(M[c], errors="coerce").fillna(0.0)
    obs = None  # 예측시 전역 obs이 없다고 가정(있으면 넣어서 perf 계산 가능)
    mperf = _global_media_perf_from_obs(obs)
    M = M.merge(mperf, on="mda_idx", how="left").fillna(0.0)
    _, CAT2DOM = _make_domain_mapping(M)

    # 상위+랜덤 샘플링
    if "glob_pop_z" in M.columns and len(M)>0:
        M_sorted = M.sort_values("glob_pop_z", ascending=False).reset_index(drop=True)
    else:
        M_sorted = M.sample(frac=1.0, random_state=42).reset_index(drop=True)
    top_cut = int(max(1, round(mda_per_ad*top_mda_frac)))
    top_ids = M_sorted["mda_idx"].head(top_cut).to_numpy()
    rest    = np.setdiff1d(M["mda_idx"].to_numpy(), top_ids)
    rnd     = np.random.RandomState(42).choice(rest, size=min(mda_per_ad-len(top_ids), len(rest)), replace=False) if len(rest)>0 else np.array([],dtype=int)
    pick    = np.concatenate([top_ids, rnd]) if len(top_ids)>0 else rnd
    grid    = M.loc[M["mda_idx"].isin(pick)].copy()
    grid.insert(0, "ads_idx", int(a.iloc[0]["ads_idx"]))
    grid.insert(1, "ads_category", a.iloc[0]["ads_category"])

    # domain 적합도
    def _dom_val(row):
        col = CAT2DOM.get(int(a.iloc[0]["ads_category"])) if pd.notna(a.iloc[0]["ads_category"]) else None
        return float(row.get(col, 0.0)) if col in row.index else 0.0
    grid["domain_fit_simple"] = grid.apply(_dom_val, axis=1)

    # sanitize 매핑 적용
    grid = grid.rename(columns=ren_map)
    X = ensure_feature_matrix(grid, feat_cols_safe)
    s = ranker.predict(X)
    out = grid[["ads_idx","mda_idx"]].copy()
    out["score"] = s
    return out.sort_values("score", ascending=False).head(10).reset_index(drop=True)

def offline_eval_on_holdout_ultra_safe(ranker, feat_cols_safe, ren_map, df_attr, valid_ads, tgt, ks=(5,10,20)):
    ads_with_gt = set(tgt.loc[_to_num(tgt["rpt_time_turn"])>0, "ads_idx"].unique().tolist())
    eval_ads = sorted(valid_ads & ads_with_gt)
    if not eval_ads:
        print("[Attribute-Only Evaluation - Ultra]\nn_ads_eval: 0"); return
    tg = (tgt.copy()
            .assign(turn=_to_num(tgt.get("rpt_time_turn", 0)))
            .groupby(["ads_idx","mda_idx"], as_index=False)
            .agg(tgt_turn=("turn","sum")))
    gts = {int(ad): {int(r.mda_idx): float(r.tgt_turn) for _, r in g.iterrows()}
           for ad, g in tg.groupby("ads_idx")}
    preds = {}
    for ad in eval_ads:
        grid = df_attr[df_attr["ads_idx"]==ad].copy().rename(columns=ren_map)
        X = ensure_feature_matrix(grid, feat_cols_safe)
        s = ranker.predict(X)
        order = np.argsort(-s)[:20]
        preds[ad] = grid.iloc[order]["mda_idx"].astype(int).tolist()
    summary, _ = evaluate_rankings(preds, gts, ks=ks)
    print("\n[Attribute-Only Evaluation - Ultra]")
    for k,v in summary.items():
        print(f"{k}: {v}")

# ---------- one-shot runner ----------
def run_all_ultra_safe(
    tgt_days=7, mda_per_ad=150, top_mda_frac=0.25, ads_limit=None, valid_ratio=0.2
):
    # 0) 월 분할
    obs, tgt, s, e, cut = split_obs_tgt_month(ads_time, tgt_days=tgt_days)
    print(f"[Window] OBS {s.date()}~{cut.date()} | TGT {(cut+pd.Timedelta(days=1)).date()}~{e.date()}")

    # 1) 속성 기반 샘플링(교차곱 생성 안함)
    df_attr, feat_cols = build_attribute_pairs_ultra_safe(
        ads_list=ads_list, media_portfolio=media_portfolio,
        obs=obs, tgt=tgt, mda_per_ad=mda_per_ad, top_mda_frac=top_mda_frac,
        ads_limit=ads_limit, rng_seed=42
    )
    print("n_pairs_attr:", len(df_attr))
    print("raw feat_cols_attr:", len(feat_cols))

    # 2) 학습(경량, crash-safe)
    ranker, valid_ads, feat_cols_safe, ren_map = train_attr_ranker_ultra_safe(
        df_attr, feat_cols, tgt, valid_ratio=valid_ratio, random_state=42
    )

    # 3) 오프라인 평가
    offline_eval_on_holdout_ultra_safe(
        ranker, feat_cols_safe, ren_map, df_attr, valid_ads, tgt, ks=(5,10,20)
    )

    # 4) 예시 예측 1건
    some_ad = next(iter(valid_ads))
    top10 = attr_predict_for_ad_ultra_safe(
        ranker, feat_cols_safe, ren_map, some_ad, media_portfolio, ads_list,
        mda_per_ad=mda_per_ad, top_mda_frac=top_mda_frac
    )
    print(top10)
    return ranker, feat_cols_safe, ren_map

# ===== 실행 예시 =====
# ranker_attr, feat_cols_attr, REN_ATTR = run_all_ultra_safe(
#     tgt_days=7, mda_per_ad=120, top_mda_frac=0.3, ads_limit=3000, valid_ratio=0.2
# )


In [24]:
ranker_attr, feat_cols_attr, REN_ATTR = run_all_ultra_safe(
    tgt_days=7, mda_per_ad=120, top_mda_frac=0.3, ads_limit=3000, valid_ratio=0.2
)

[Window] OBS 2025-08-01~2025-08-24 | TGT 2025-08-25~2025-08-31
n_pairs_attr: 360000
raw feat_cols_attr: 54
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=2.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.0
[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=2.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.0
[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current v

# ---- 최소 피처 셋(없으면 0 채움; ensure_feature_matrix가 안전 처리) ----
FEAT_MIN = [
    # 매체 전반(OBS 집계)
    "mda_clk_global",          # OBS 전체 클릭 수 (규모/활동성 proxy)
    "mda_conv_global",         # OBS 전체 전환 수
    "mda_cvr_global",          # OBS 전환율 = conv/clk
    "mda_pop_norm",            # 매체 전반 인기(표준화 conv)

    # 카테고리 × 매체 적합도(OBS 집계)
    "cat_cvr",                 # 해당 광고 카테고리에서의 매체 CVR (conv/clk)
    "cat_share",               # 매체 전체 전환 중 해당 카테고리 비중

    # 광고 가격/속성(ads_list)
    "price_reward_log",        # log(ads_reward_price+1)
    "price_contract_log",      # log(ads_contract_price+1)
    "price_ratio",             # reward/contract (NaN→0)

    # media_portfolio(있으면 사용, 없으면 0)
    "mda_size",                # 규모(예: 소/중/대 수치화 or 점수)
    "active_ratio",            # 활성 지표
    "day_ratio", "weekend_ratio"
]

In [28]:
# ============================================================
#  Ad × Media 추천 (Attribute-only: 콜드스타트 안전 버전)
#  - 과거 "광고×매체" 페어 성과/CF 제거
#  - 최소 피처 기반 LambdaMART 파이프라인
# ============================================================

# 경고 최소화
warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

# -------------------------------
# 0) 공통 유틸
# -------------------------------
def _to_num(s): 
    return pd.to_numeric(s, errors="coerce").fillna(0)

def _to_dt(s):
    return pd.to_datetime(s, errors="coerce")

# -------------------------------
# 1) 관측/타깃 윈도우 (최신 달 기준, 타깃 7일)
# -------------------------------
def split_obs_tgt_month(ads_time: pd.DataFrame, tgt_days: int = 7):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    last_date = t["date"].max()
    if pd.isna(last_date):
        raise ValueError("ads_time['rpt_time_date']에서 유효한 날짜를 찾을 수 없습니다.")
    month_start = last_date.normalize().replace(day=1)
    next_month = (month_start + pd.offsets.MonthBegin(1))
    month_end = (next_month - pd.Timedelta(days=1)).normalize()
    cutoff = month_end - pd.Timedelta(days=tgt_days)
    obs = t[(t["date"] >= month_start) & (t["date"] <= cutoff)].copy()
    tgt = t[(t["date"] >  cutoff) & (t["date"] <= month_end)].copy()
    return obs, tgt, month_start, month_end, cutoff

# -------------------------------
# 2) 전역 속성 신호 (OBS만 사용)
#    - 매체 전반: 클릭/전환/전환율/표준화된 인기
#    - 카테고리×매체: 해당 카테고리에서의 CVR, 카테고리 비중
# -------------------------------
def build_attr_context(obs: pd.DataFrame, ads_list: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # 매체 전반 집계
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"),
                      mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0, np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std()
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 카테고리×매체 집계
    al = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx")
    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"] / cat_mda["clk"].replace(0, np.nan)).fillna(0.0)

    # 매체 전체 전환 대비 해당 카테고리 비중
    total_conv = mda_all.set_index("mda_idx")["mda_conv_global"].to_dict()
    cat_mda["cat_share"] = [
        (c / total_conv.get(int(m), 1.0)) if total_conv.get(int(m), 0.0) > 0 else 0.0
        for m, c in zip(cat_mda["mda_idx"], cat_mda["conv"])
    ]

    return {"mda_all": mda_all, "cat_mda": cat_mda}

# -------------------------------
# 3) 랭커 학습용 데이터셋 (속성 기반 후보 생성)
#    - 후보: 전반 인기 상위 + 랜덤 탐색
#    - 금지: 광고×매체 과거 페어 성과, CF 임베딩
# -------------------------------
# --- 새 옵션: 후보/광고 축소 ---


def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = None,
                            rng_seed: int = 42) -> np.ndarray:
    """tgt에서 양성(mda 전환>0) 개수가 min_pos 이상인 광고만 선택, 필요시 랜덤 샘플링"""
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1, int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

def build_ranker_dataset_attr_only(
    obs: pd.DataFrame, tgt: pd.DataFrame, ads_list: pd.DataFrame,
    topC: int = 80, add_random: int = 40,               # ↓↓↓ 기본값을 대폭 축소
    mp_add: pd.DataFrame | None = None,
    # --- 새 인자들 ---
    train_ads_filter: Optional[Iterable[int]] = None,   # 학습에 쓸 광고 리스트 (없으면 전체)
    cat_topM: Optional[int] = 60                        # 카테고리 상위 매체만 우선 후보 (None이면 비활성)
):
    actx = build_attr_context(obs, ads_list)
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]

    if mda_all.empty:
        raise ValueError("관측 구간(OBS)에서 매체 집계가 비어 있습니다.")

    # 학습 광고 필터
    all_ad_ids = ads_list["ads_idx"].dropna().astype(int).unique()
    if train_ads_filter is not None:
        ad_ids = np.intersect1d(all_ad_ids, np.array(list(train_ads_filter), dtype=int))
    else:
        ad_ids = all_ad_ids

    # 글로벌 인기 기반 정렬
    mda_ids_global = mda_all.sort_values("mda_conv_global", ascending=False)["mda_idx"].astype(int).values
    rng = np.random.RandomState(42)

    # 카테고리별 상위 매체 캐시 (옵션)
    cat_top_cache = {}
    if cat_topM is not None:
        for cat, g in cat_mda.groupby("ads_category"):
            cand = g.sort_values("cat_cvr", ascending=False)["mda_idx"].astype(int).head(cat_topM).values
            cat_top_cache[int(cat) if pd.notna(cat) else -1] = cand

    # 후보 생성 (카테고리 우선 + 글로벌 보정 + 랜덤 소량)
    cand_rows = []
    ad_meta = ads_list[["ads_idx","ads_category","ads_reward_price","ads_contract_price"]].drop_duplicates("ads_idx")
    ad_meta["ads_idx"] = ad_meta["ads_idx"].astype(int)
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    for ad in ad_ids:
        base = []
        # 1) 카테고리 상위
        cat = ad2cat.get(int(ad), np.nan)
        if cat_topM is not None:
            top_by_cat = cat_top_cache.get(int(cat) if pd.notna(cat) else -1, np.array([], dtype=int))
            base.extend(top_by_cat.tolist())
        # 2) 글로벌 인기 보정
        base.extend(mda_ids_global[:min(topC, len(mda_ids_global))].tolist())
        base = list(dict.fromkeys(base))  # 중복 제거

        # 3) 랜덤 소량
        remain = np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain) > 0 and add_random > 0:
            rnd = rng.choice(remain, size=min(add_random, len(remain)), replace=False).tolist()
            base.extend(rnd)
        base = list(dict.fromkeys(base))

        cand_rows += [(int(ad), int(m)) for m in base]

    df = pd.DataFrame(cand_rows, columns=["ads_idx","mda_idx"]).drop_duplicates()

    # ---- 조인/파생 (기존과 동일) ----
    df = df.merge(mda_all, on="mda_idx", how="left")

    # 광고 속성 파생
    ad_meta = ad_meta.copy()
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = pd.to_numeric(ad_meta[c], errors="coerce").fillna(0.0)
    ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
    ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                     ad_meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)
    df = df.merge(ad_meta[["ads_idx","ads_category","price_reward_log",
                           "price_contract_log","price_ratio"]], on="ads_idx", how="left")

    # 카테고리×매체 적합도
    df = df.merge(cat_mda.rename(columns={"clk":"cat_clk","conv":"cat_conv"}),
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0, "cat_share":0.0})

    # media_portfolio (선택)
    if mp_add is not None and not mp_add.empty:
        keep_mp = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp_add.columns]
        if keep_mp:
            df = df.merge(mp_add[keep_mp], on="mda_idx", how="left").fillna(0.0)

    # 타깃 라벨
    tgt_g = (tgt.groupby(["ads_idx","mda_idx"], as_index=False)
               .agg(tgt_turn=("rpt_time_turn", lambda s: _to_num(s).sum())))
    df = df.merge(tgt_g, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_turn":0.0})

    # relevance binning
    df = make_relevance_balanced(df, n_bins=3)
    return df, actx


# -------------------------------
# 4) 라벨/다운샘플/가중치
# -------------------------------
LABEL_GAIN = [0.0, 1.0, 4.0, 12.0]  # 0(neg), 1,2,3 bin에 대한 gain

def make_relevance_balanced(df_rank, n_bins=3):
    df = df_rank.copy()
    y  = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum() > 0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=n_bins, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int) + 1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df

def downsample_negatives_per_ad(df, ratio=8, hard_top=80, hard_by="mda_pop_norm", max_neg=400, random_state=42):
    out = []; rng = np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"] > 0]; neg = g[g["tgt_turn"] <= 0]
        if len(pos) == 0:  # 학습에 안 쓰는 광고
            continue
        if (hard_by in neg.columns) and (len(neg) > 0):
            hard = neg.sort_values(hard_by, ascending=False).head(hard_top); rest = neg.drop(hard.index)
        else:
            hard = pd.DataFrame([], columns=neg.columns); rest = neg
        cap = min(max_neg, ratio * len(pos))
        need_rand = max(0, cap - len(hard))
        if need_rand > 0 and len(rest) > 0:
            rand = rest.sample(n=min(need_rand, len(rest)), random_state=rng)
            neg_keep = pd.concat([hard, rand], ignore_index=True)
        else:
            neg_keep = hard
        out.append(pd.concat([pos, neg_keep], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

# -------------------------------
# 5) 안전한 피처 행렬 (최소 피처만)
# -------------------------------
FEAT_MIN = [
    # 매체 전반
    "mda_cvr_global",   # 매체 전환율
    "mda_pop_norm",     # 매체 전환수 기반 표준화된 인기

    # 카테고리 × 매체
    "cat_cvr",          # 해당 카테고리에서 매체의 전환율

    # 광고 속성
    "price_reward_log", # log(리워드 가격)
    "price_ratio"       # reward / contract 비율
]


def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns:
            X[c] = 0.0
    X = X[feat_cols].copy()
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors="coerce")
    return X.fillna(0.0).astype(np.float32)

# -------------------------------
# 6) 평가지표
# -------------------------------
def precision_at_k(gt_set, pred_list, k):
    if k <= 0: return np.nan
    k = min(k, len(pred_list))
    return sum(1 for a in pred_list[:k] if a in gt_set) / max(1, k)

def recall_at_k(gt_set, pred_list, k):
    if len(gt_set) == 0: return np.nan
    return sum(1 for a in pred_list[:k] if a in gt_set) / len(gt_set)

def average_precision_at_k(gt_set, pred_list, k):
    if len(gt_set) == 0: return np.nan
    hits, ap = 0, 0.0
    for i, a in enumerate(pred_list[:k], start=1):
        if a in gt_set:
            hits += 1; ap += hits / i
    return ap / min(len(gt_set), k)

def ndcg_at_k_from_gains(gains_in_pred_order, k):
    k = min(k, len(gains_in_pred_order))
    dcg  = sum(g / math.log2(i+2) for i, g in enumerate(gains_in_pred_order[:k]))
    ideal_gains = sorted(gains_in_pred_order, reverse=True)[:k]
    idcg = sum(g / math.log2(i+2) for i, g in enumerate(ideal_gains))
    return dcg / idcg if idcg > 0 else 0.0

def ndcg_at_k(gt_gain_map, pred_list, k):
    gains = [gt_gain_map.get(a, 0.0) for a in pred_list[:k]]
    return ndcg_at_k_from_gains(gains, k)

def hit_rate_at_k(gt_set, pred_list, k):
    if len(gt_set) == 0: return np.nan
    return 1.0 if any(a in gt_set for a in pred_list[:k]) else 0.0

def evaluate_rankings(pred_dict, gt_dict, ks=(5,10,20), use_gain=True):
    rows = []
    for ad, preds in pred_dict.items():
        if ad not in gt_dict: continue
        gt = gt_dict[ad]
        if isinstance(gt, dict):
            gt_set = set(gt.keys()); gt_gain_map = gt
        else:
            gt_set = set(gt); gt_gain_map = {x: 1.0 for x in gt_set}
        if len(preds) == 0: continue
        row = {"ads_idx": ad, "gt_size": len(gt_set)}
        for k in ks:
            row[f"Precision@{k}"] = precision_at_k(gt_set, preds, k)
            row[f"Recall@{k}"]    = recall_at_k(gt_set, preds, k)
            row[f"MAP@{k}"]       = average_precision_at_k(gt_set, preds, k)
            row[f"NDCG@{k}"]      = ndcg_at_k(gt_gain_map if use_gain else {m:1.0 for m in gt_set}, preds, k)
            row[f"HitRate@{k}"]   = hit_rate_at_k(gt_set, preds, k)
        rows.append(row)
    per_ad = pd.DataFrame(rows)
    summary = {"n_ads_eval": int(per_ad.shape[0])}
    for k in ks:
        for m in ["Precision", "Recall", "MAP", "NDCG", "HitRate"]:
            col = f"{m}@{k}"
            summary[col] = float(per_ad[col].mean(skipna=True)) if col in per_ad else np.nan
    summary["avg_gt_size"] = float(per_ad["gt_size"].mean(skipna=True)) if "gt_size" in per_ad else np.nan
    return summary, per_ad

def gts_gain_from_tgt(tgt: pd.DataFrame):
    g = (tgt.groupby(["ads_idx","mda_idx"], as_index=False)
           .agg(turn=("rpt_time_turn", lambda s: _to_num(s).sum())))
    out = {int(ad): {int(r["mda_idx"]): float(r["turn"]) for _, r in gg.iterrows()}
           for ad, gg in g.groupby("ads_idx")}
    return out

# -------------------------------
# 7) 예측(속성 기반: 페어 성과/CF 금지)
# -------------------------------
def predict_all_mda_attr_only(
    ranker, feat_cols, obs: pd.DataFrame, ads_list: pd.DataFrame,
    K: int = 20, mp_add: pd.DataFrame | None = None
):
    actx = build_attr_context(obs, ads_list)
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]
    mda_ids = mda_all["mda_idx"].astype(int).values

    def _grid_for_ad(ad):
        g = pd.DataFrame({"ads_idx": int(ad), "mda_idx": mda_ids})
        g = g.merge(mda_all, on="mda_idx", how="left")

        # 광고 속성
        meta = ads_list.loc[ads_list["ads_idx"]==ad, ["ads_idx","ads_category","ads_reward_price","ads_contract_price"]].drop_duplicates("ads_idx")
        if meta.empty:
            meta = pd.DataFrame([{
                "ads_idx": int(ad), "ads_category": np.nan,
                "ads_reward_price": 0.0, "ads_contract_price": 0.0
            }])
        meta["ads_reward_price"]   = pd.to_numeric(meta["ads_reward_price"], errors="coerce").fillna(0.0)
        meta["ads_contract_price"] = pd.to_numeric(meta["ads_contract_price"], errors="coerce").fillna(0.0)
        meta["price_reward_log"]   = np.log1p(meta["ads_reward_price"])
        meta["price_contract_log"] = np.log1p(meta["ads_contract_price"])
        meta["price_ratio"]        = (meta["ads_reward_price"] /
                                      meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)
        g = g.merge(meta[["ads_idx","ads_category","price_reward_log","price_contract_log","price_ratio"]],
                    on="ads_idx", how="left")

        # 카테고리×매체 적합도
        g = g.merge(cat_mda.rename(columns={"clk":"cat_clk","conv":"cat_conv"}),
                    on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0, "cat_share":0.0})

        # media_portfolio (선택)
        if mp_add is not None and not mp_add.empty:
            keep_mp = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp_add.columns]
            if keep_mp:
                g = g.merge(mp_add[keep_mp], on="mda_idx", how="left").fillna(0.0)
        return g

    preds = {}
    for ad in ads_list["ads_idx"].dropna().astype(int).unique():
        grid = _grid_for_ad(int(ad))
        X = ensure_feature_matrix(grid, feat_cols)
        score = ranker.predict(X)
        top_idx = np.argsort(-score)[:min(K, len(score))]
        preds[int(ad)] = grid.iloc[top_idx]["mda_idx"].astype(int).tolist()
    return preds

# ============================================================
# ==============   메인 실행부 (예시)   =======================
# ============================================================
# 준비: ads_time, ads_list, media_portfolio 가 메모리에 있다고 가정
# - media_portfolio에서 사용할 최소 컬럼만 numeric으로 캐스팅

# 0) media_portfolio 전처리 (선택)
drop_mp = {"Unnamed: 0", "first_click", "last_click", "expected_total_profit"}
feat_mp = [c for c in getattr(media_portfolio, "columns", []) if c not in drop_mp and c != "mda_idx"]
mp_add = None
if "mda_idx" in getattr(media_portfolio, "columns", []):
    mp_add = media_portfolio[["mda_idx"] + feat_mp].copy()
    for c in feat_mp:
        mp_add[c] = pd.to_numeric(mp_add[c], errors="coerce").fillna(0.0)

# (A) 윈도우 분할
obs, tgt, start, end, cut = split_obs_tgt_month(ads_time, tgt_days=7)
print(f"[Window] OBS: {start.date()} → {cut.date()}  |  TGT: {(cut+pd.Timedelta(days=1)).date()} → {end.date()}")

# --- (0) 학습 대상 광고 축소: tgt에서 양성 있는 광고 + 상한 ---
train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=2, max_train_ads=8000)

# --- (B) 속성 기반 랭킹 데이터셋 (후보 축소 옵션 활성화) ---
df_rank, _ = build_ranker_dataset_attr_only(
    obs, tgt, ads_list,
    topC=80, add_random=40,          # 후보 축소
    mp_add=mp_add,
    train_ads_filter=train_ads,      # 학습 광고만 후보 생성
    cat_topM=60                      # 카테고리 상위 매체 우선
)

# --- (C) Train/Valid 스플릿 (학습 광고에서만 스플릿) ---
ads_all  = np.array(train_ads, dtype=int)
rng      = np.random.RandomState(42); rng.shuffle(ads_all)
valid_ads = set(ads_all[: int(len(ads_all)*0.2)])
df_rank["is_valid"] = df_rank["ads_idx"].astype(int).isin(valid_ads)
tr = df_rank[~df_rank["is_valid"]].copy()
va = df_rank[ df_rank["is_valid"]].copy()
print("train size:", len(tr), "valid size:", len(va))
print("train relevance dist:\n", tr["relevance"].value_counts().sort_index())
print("valid  relevance dist:\n", va["relevance"].value_counts().sort_index())

# --- (D) 다운샘플 더 강하게 ---
tr = downsample_negatives_per_ad(tr, ratio=4, hard_top=40, hard_by="mda_pop_norm", max_neg=120)
tr = tr.sort_values(["ads_idx"]); va = va.sort_values(["ads_idx"])
group_tr = tr.groupby("ads_idx").size().astype(int).tolist()
group_va = va.groupby("ads_idx").size().astype(int).tolist()

# --- (E) 피처/가중치/학습 ---
# FEAT_MIN는 초간단 5개 버전으로 이미 위에서 설정했다고 가정
drop_cols = ["tgt_turn","is_valid","relevance","ads_category"]
base_cols = ["ads_idx","mda_idx"]
feat_cols = [c for c in FEAT_MIN if c not in set(drop_cols + base_cols)]

X_tr = ensure_feature_matrix(tr, feat_cols)
X_va = ensure_feature_matrix(va, feat_cols)
y_tr = tr["relevance"].astype(int)
y_va = va["relevance"].astype(int)

w_tr = np.ones(len(tr), dtype=float)
w_va = np.ones(len(va), dtype=float)
if "mda_clk_global" in tr.columns:
    w_tr = np.clip(tr["mda_clk_global"].values, 1.0, None)
    w_va = np.clip(va["mda_clk_global"].values, 1.0, None)

ranker = lgb.LGBMRanker(
    objective="lambdarank", metric="ndcg", eval_at=[5,10,20],
    learning_rate=0.03, n_estimators=2000, num_leaves=47,
    min_data_in_leaf=80, lambda_l2=2.0, subsample=0.9,
    colsample_bytree=0.9, bagging_freq=1, feature_fraction=0.9,
    label_gain=LABEL_GAIN, random_state=42, n_jobs=-1, verbosity=-1
)
ranker.fit(
    X_tr, y_tr, sample_weight=w_tr, group=group_tr,
    eval_set=[(X_va, y_va)], eval_group=[group_va],
    eval_sample_weight=[w_va],
    callbacks=[lgb.early_stopping(200), lgb.log_evaluation(100)]
)
print("best_iteration_", getattr(ranker, "best_iteration_", None))

# (F) 예측 + 평가
preds = predict_all_mda_attr_only(ranker, feat_cols, obs, ads_list, K=20, mp_add=mp_add)
gts = gts_gain_from_tgt(tgt)
ads_eval = sorted(set(preds) & set(gts))
summary, per_ad = evaluate_rankings({a: preds[a] for a in ads_eval},
                                    {a: gts[a]   for a in ads_eval},
                                    ks=(5,10,20), use_gain=True)

print("\n[Offline Evaluation Summary]")
for k, v in summary.items():
    print(f"{k}: {v}")

if len(ads_eval) > 0:
    some_ad = ads_eval[0]
    print(f"\n[Example Recommendations for ads_idx={some_ad}] Top-10 mda:")
    print(pd.DataFrame({"mda_idx": preds[some_ad][:10]}))


[Window] OBS: 2025-08-01 → 2025-08-24  |  TGT: 2025-08-25 → 2025-08-31
train size: 30871 valid size: 7646
train relevance dist:
 relevance
0    30085
1      300
2      245
3      241
Name: count, dtype: int64
valid  relevance dist:
 relevance
0    7476
1      52
2      45
3      73
Name: count, dtype: int64
Training until validation scores don't improve for 200 rounds
[100]	valid_0's ndcg@5: 0.784426	valid_0's ndcg@10: 0.802338	valid_0's ndcg@20: 0.815269
[200]	valid_0's ndcg@5: 0.776636	valid_0's ndcg@10: 0.806358	valid_0's ndcg@20: 0.812321
[300]	valid_0's ndcg@5: 0.773934	valid_0's ndcg@10: 0.801924	valid_0's ndcg@20: 0.810465
Early stopping, best iteration is:
[112]	valid_0's ndcg@5: 0.786986	valid_0's ndcg@10: 0.813885	valid_0's ndcg@20: 0.821989
best_iteration_ 112


KeyboardInterrupt: 

In [37]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import warnings
import math
from typing import Optional, Dict, List, Tuple, Any
from datetime import datetime, timedelta

warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

class AdMediaRecommendationSystem:
    """
    광고 매체사 추천 시스템
    - 속성 기반 모델 (과거 성과 데이터 제외)
    - LambdaMART 기반 랭킹 모델
    - 콜드스타트 문제 해결
    """
    
    def __init__(self, target_days: int = 7, random_state: int = 42):
        self.target_days = target_days
        self.random_state = random_state
        self.ranker = None
        self.feature_columns = None
        self.context_data = None
        
        # 최소 피처 세트 (속성 기반)
        self.MINIMAL_FEATURES = [
            "mda_cvr_global",    # 매체 전환율
            "mda_pop_norm",      # 매체 인기도 (정규화)
            "cat_cvr",           # 카테고리별 매체 전환율
            "price_reward_log",  # 광고 리워드 가격 (로그)
            "price_ratio"        # 가격 비율
        ]
        
        # 라벨 게인 (relevance binning용)
        self.LABEL_GAIN = [0.0, 1.0, 4.0, 12.0]
    
    def _to_numeric_safe(self, series):
        """안전한 숫자 변환"""
        if isinstance(series, (list, np.ndarray)):
            series = pd.Series(series)
        elif not isinstance(series, pd.Series):
            series = pd.Series([series])
        return pd.to_numeric(series, errors="coerce").fillna(0)
    
    def _to_datetime_safe(self, series):
        """안전한 날짜 변환"""
        if isinstance(series, (list, np.ndarray)):
            series = pd.Series(series)
        elif not isinstance(series, pd.Series):
            series = pd.Series([series])
        return pd.to_datetime(series, errors="coerce")
    
    def split_time_windows(self, ads_time: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, dict]:
        """
        시간 윈도우 분할: 관측 구간 vs 타겟 구간
        """
        df = ads_time.copy()
        df["date"] = self._to_datetime_safe(df["rpt_time_date"])
        
        last_date = df["date"].max()
        if pd.isna(last_date):
            raise ValueError("유효한 날짜 데이터가 없습니다.")
        
        # 실제 데이터의 날짜 범위 기준으로 분할
        first_date = df["date"].min()
        
        print(f"📅 실제 데이터 범위: {first_date.date()} ~ {last_date.date()}")
        
        # 전체 기간에서 마지막 target_days를 타겟으로, 나머지를 관측으로 분할
        cutoff_date = last_date - pd.Timedelta(days=self.target_days)
        
        obs_data = df[(df["date"] >= first_date) & (df["date"] <= cutoff_date)].copy()
        target_data = df[(df["date"] > cutoff_date) & (df["date"] <= last_date)].copy()
        
        window_info = {
            "data_start": first_date,
            "data_end": last_date,
            "obs_start": first_date,
            "obs_end": cutoff_date,
            "target_start": cutoff_date + pd.Timedelta(days=1),
            "target_end": last_date
        }
        
        return obs_data, target_data, window_info
    
    def build_context_features(self, obs_data: pd.DataFrame, ads_list: pd.DataFrame) -> Dict[str, pd.DataFrame]:
        """
        전역 컨텍스트 피처 생성 (속성 기반)
        """
        df = obs_data.copy()
        df["clicks"] = pd.to_numeric(df.get("rpt_time_clk", 0), errors="coerce").fillna(0)
        df["conversions"] = pd.to_numeric(df.get("rpt_time_turn", 0), errors="coerce").fillna(0)
        
        # 1. 매체별 전반적 성과
        media_stats = (df.groupby("mda_idx", as_index=False)
                      .agg(
                          total_clicks=("clicks", "sum"),
                          total_conversions=("conversions", "sum")
                      ))
        
        # 매체 전환율 계산
        media_stats["mda_cvr_global"] = (
            media_stats["total_conversions"] / 
            media_stats["total_clicks"].replace(0, np.nan)
        ).fillna(0.0)
        
        # 매체 인기도 정규화
        conv_mean = media_stats["total_conversions"].mean()
        conv_std = media_stats["total_conversions"].std()
        media_stats["mda_pop_norm"] = (
            (media_stats["total_conversions"] - conv_mean) / (conv_std + 1e-9)
        )
        
        # 2. 카테고리 x 매체 성과
        ads_meta = ads_list[["ads_idx", "ads_category"]].drop_duplicates("ads_idx")
        df_with_cat = df.merge(ads_meta, on="ads_idx", how="left")
        
        category_media_stats = (df_with_cat.groupby(["mda_idx", "ads_category"], as_index=False)
                              .agg(
                                  cat_clicks=("clicks", "sum"),
                                  cat_conversions=("conversions", "sum")
                              ))
        
        category_media_stats["cat_cvr"] = (
            category_media_stats["cat_conversions"] / 
            category_media_stats["cat_clicks"].replace(0, np.nan)
        ).fillna(0.0)
        
        return {
            "media_stats": media_stats,
            "category_media_stats": category_media_stats
        }
    
    def create_candidates(self, ads_list: pd.DataFrame, media_stats: pd.DataFrame,
                         top_global: int = 50, add_random: int = 20) -> pd.DataFrame:
        """
        후보 (광고, 매체) 페어 생성 - 속성 기반
        """
        # 전역 인기 순 매체 ID
        top_media_ids = (media_stats.sort_values("total_conversions", ascending=False)
                        ["mda_idx"].head(top_global).values)
        
        # 랜덤 추가 매체
        remaining_media = media_stats[~media_stats["mda_idx"].isin(top_media_ids)]["mda_idx"].values
        if len(remaining_media) > 0:
            rng = np.random.RandomState(self.random_state)
            random_media = rng.choice(remaining_media, 
                                    size=min(add_random, len(remaining_media)), 
                                    replace=False)
            candidate_media = np.concatenate([top_media_ids, random_media])
        else:
            candidate_media = top_media_ids
        
        # 모든 광고 x 후보 매체 조합 생성
        ad_ids = ads_list["ads_idx"].dropna().unique()
        candidates = []
        
        for ad_id in ad_ids:
            for media_id in candidate_media:
                candidates.append({"ads_idx": int(ad_id), "mda_idx": int(media_id)})
        
        return pd.DataFrame(candidates)
    
    def prepare_training_data(self, obs_data: pd.DataFrame, target_data: pd.DataFrame,
                            ads_list: pd.DataFrame, max_train_ads: Optional[int] = None) -> pd.DataFrame:
        """
        학습 데이터셋 준비
        """
        # 컨텍스트 피처 생성
        self.context_data = self.build_context_features(obs_data, ads_list)
        media_stats = self.context_data["media_stats"]
        category_stats = self.context_data["category_media_stats"]
        
        # 학습할 광고 선택 (타겟 데이터에서 전환이 있는 광고 위주)
        target_conversions = (target_data.groupby("ads_idx")["rpt_time_turn"]
                            .apply(lambda x: pd.to_numeric(x, errors="coerce").fillna(0).sum())
                            .reset_index())
        
        train_ads = target_conversions[target_conversions["rpt_time_turn"] > 0]["ads_idx"].values
        if max_train_ads and len(train_ads) > max_train_ads:
            rng = np.random.RandomState(self.random_state)
            train_ads = rng.choice(train_ads, size=max_train_ads, replace=False)
        
        # 후보 생성
        candidates = self.create_candidates(
            ads_list[ads_list["ads_idx"].isin(train_ads)], 
            media_stats
        )
        
        # 피처 조인
        # 1) 매체 피처
        candidates = candidates.merge(media_stats[["mda_idx", "mda_cvr_global", "mda_pop_norm"]], 
                                    on="mda_idx", how="left")
        
        # 2) 광고 피처
        ads_features = ads_list[["ads_idx", "ads_category", "ads_reward_price", "ads_contract_price"]].copy()
        for col in ["ads_reward_price", "ads_contract_price"]:
            if col in ads_features.columns:
                ads_features[col] = pd.to_numeric(ads_features[col], errors="coerce").fillna(0)
            else:
                ads_features[col] = 0
        
        ads_features["price_reward_log"] = np.log1p(ads_features["ads_reward_price"])
        ads_features["price_ratio"] = (
            ads_features["ads_reward_price"] / 
            ads_features["ads_contract_price"].replace(0, np.nan)
        ).fillna(0.0)
        
        candidates = candidates.merge(
            ads_features[["ads_idx", "ads_category", "price_reward_log", "price_ratio"]], 
            on="ads_idx", how="left"
        )
        
        # 3) 카테고리 x 매체 피처
        candidates = candidates.merge(
            category_stats[["mda_idx", "ads_category", "cat_cvr"]], 
            on=["mda_idx", "ads_category"], how="left"
        ).fillna({"cat_cvr": 0.0})
        
        # 4) 타겟 라벨 (실제 전환수)
        target_labels = (target_data.groupby(["ads_idx", "mda_idx"], as_index=False)
                        .agg(target_conversions=("rpt_time_turn", 
                                               lambda x: pd.to_numeric(x, errors="coerce").fillna(0).sum())))
        
        candidates = candidates.merge(target_labels, on=["ads_idx", "mda_idx"], how="left")
        candidates["target_conversions"] = candidates["target_conversions"].fillna(0.0)
        
        # 5) Relevance binning
        candidates = self._create_relevance_bins(candidates)
        
        return candidates
    
    def _create_relevance_bins(self, df: pd.DataFrame, n_bins: int = 3) -> pd.DataFrame:
        """
        연속적인 전환수를 relevance bin으로 변환
        """
        df = df.copy()
        conversions = df["target_conversions"].astype(float)
        relevance = np.zeros(len(df), dtype=int)
        
        positive_mask = conversions > 0
        if positive_mask.sum() > 0:
            log_conversions = np.log1p(conversions[positive_mask])
            try:
                bins = pd.qcut(log_conversions, q=n_bins, labels=False, duplicates="drop")
                relevance[positive_mask] = bins.astype(int) + 1
            except ValueError:
                relevance[positive_mask] = 1
        
        df["relevance"] = relevance
        return df
    
    def downsample_negatives(self, df: pd.DataFrame, neg_ratio: int = 5) -> pd.DataFrame:
        """
        네거티브 샘플 다운샘플링 (광고별로)
        """
        result_dfs = []
        rng = np.random.RandomState(self.random_state)
        
        for ad_id, group in df.groupby("ads_idx"):
            positives = group[group["target_conversions"] > 0]
            negatives = group[group["target_conversions"] <= 0]
            
            if len(positives) == 0:
                continue
            
            # 네거티브 샘플링
            max_negatives = len(positives) * neg_ratio
            if len(negatives) > max_negatives:
                negatives = negatives.sample(n=max_negatives, random_state=rng)
            
            result_dfs.append(pd.concat([positives, negatives], ignore_index=True))
        
        return pd.concat(result_dfs, ignore_index=True) if result_dfs else df
    
    def train_model(self, training_data: pd.DataFrame) -> None:
        """
        LambdaMART 모델 학습
        """
        # 피처 준비
        feature_cols = [col for col in self.MINIMAL_FEATURES if col in training_data.columns]
        self.feature_columns = feature_cols
        
        X = training_data[feature_cols].fillna(0.0).astype(np.float32)
        y = training_data["relevance"].astype(int)
        
        # 그룹 정보 (광고별)
        groups = training_data.groupby("ads_idx").size().values
        
        # 가중치 (매체 인기도 기반)
        weights = np.ones(len(training_data))
        if "mda_pop_norm" in training_data.columns:
            weights = np.exp(training_data["mda_pop_norm"].clip(-2, 2))
        
        # 모델 학습
        self.ranker = lgb.LGBMRanker(
            objective="lambdarank",
            metric="ndcg",
            eval_at=[5, 10, 20],
            learning_rate=0.05,
            n_estimators=1000,
            num_leaves=31,
            min_data_in_leaf=20,
            lambda_l2=1.0,
            subsample=0.8,
            colsample_bytree=0.8,
            label_gain=self.LABEL_GAIN,
            random_state=self.random_state,
            n_jobs=-1,
            verbosity=-1
        )
        
        self.ranker.fit(X, y, sample_weight=weights, group=groups)
        
        print(f"모델 학습 완료 - 사용된 피처: {feature_cols}")
    
    def predict_top_media(self, ads_list: pd.DataFrame, top_k: int = 10) -> Dict[int, List[Tuple[int, float]]]:
        """
        각 광고별 상위 K개 매체 추천
        """
        if self.ranker is None or self.context_data is None:
            raise ValueError("모델이 학습되지 않았습니다.")
        
        media_stats = self.context_data["media_stats"]
        category_stats = self.context_data["category_media_stats"]
        all_media_ids = media_stats["mda_idx"].values
        
        predictions = {}
        
        for _, ad_row in ads_list.iterrows():
            ad_id = int(ad_row["ads_idx"])
            
            # 모든 매체에 대한 피처 생성
            candidates = pd.DataFrame({
                "ads_idx": ad_id,
                "mda_idx": all_media_ids
            })
            
            # 매체 피처 조인
            candidates = candidates.merge(
                media_stats[["mda_idx", "mda_cvr_global", "mda_pop_norm"]], 
                on="mda_idx", how="left"
            )
            
            # 광고 피처 추가
            reward_price = float(ad_row.get("ads_reward_price", 0))
            contract_price = float(ad_row.get("ads_contract_price", 1))
            
            # 숫자 변환 처리
            if pd.isna(reward_price) or reward_price <= 0:
                reward_price = 0
            if pd.isna(contract_price) or contract_price <= 0:
                contract_price = 1
            
            candidates["price_reward_log"] = np.log1p(reward_price)
            candidates["price_ratio"] = reward_price / max(contract_price, 1.0)
            candidates["ads_category"] = ad_row.get("ads_category", "")
            
            # 카테고리 x 매체 피처 조인
            candidates = candidates.merge(
                category_stats[["mda_idx", "ads_category", "cat_cvr"]], 
                on=["mda_idx", "ads_category"], how="left"
            ).fillna({"cat_cvr": 0.0})
            
            # 예측
            X = candidates[self.feature_columns].fillna(0.0).astype(np.float32)
            scores = self.ranker.predict(X)
            
            # 상위 K개 선택
            top_indices = np.argsort(-scores)[:top_k]
            top_media_scores = [
                (int(candidates.iloc[i]["mda_idx"]), float(scores[i])) 
                for i in top_indices
            ]
            
            predictions[ad_id] = top_media_scores
        
        return predictions
    
    def calculate_predicted_conversions(self, ad_id: int, media_recommendations: List[Tuple[int, float]],
                                      budget: float = 1000000) -> Dict[int, Dict[str, float]]:
        """
        예측 전환수 계산
        """
        if self.context_data is None:
            return {}
        
        media_stats = self.context_data["media_stats"]
        results = {}
        
        for media_id, score in media_recommendations:
            media_info = media_stats[media_stats["mda_idx"] == media_id]
            if media_info.empty:
                continue
            
            media_info = media_info.iloc[0]
            
            # 가정: 매체별 CPM (Cost Per Mille)
            estimated_cpm = 10000  # 1만원/1000노출
            estimated_ctr = max(media_info["mda_cvr_global"], 0.01)  # 최소 1%
            
            # 예측 계산
            estimated_impressions = (budget / estimated_cpm) * 1000
            estimated_clicks = estimated_impressions * estimated_ctr
            estimated_conversions = estimated_clicks * estimated_ctr * score
            estimated_roi = (estimated_conversions * 50000 / budget) * 100  # 전환당 5만원 가정
            
            results[media_id] = {
                "predicted_impressions": int(estimated_impressions),
                "predicted_clicks": int(estimated_clicks),
                "predicted_conversions": int(estimated_conversions),
                "predicted_roi": round(estimated_roi, 2),
                "recommendation_score": round(score, 4)
            }
        
        return results
    
    def search_media_info(self, media_id: int, ads_time_data: pd.DataFrame = None) -> Dict[str, Any]:
        """
        매체사 ID로 매체 정보 및 전환수 검색
        """
        if self.context_data is None:
            return {"error": "모델이 학습되지 않았습니다."}
        
        media_stats = self.context_data["media_stats"]
        category_stats = self.context_data["category_media_stats"]
        
        # 매체 기본 정보
        media_info = media_stats[media_stats["mda_idx"] == media_id]
        if media_info.empty:
            return {"error": f"매체 ID {media_id}를 찾을 수 없습니다."}
        
        media_info = media_info.iloc[0]
        
        # 카테고리별 성과
        cat_performance = category_stats[category_stats["mda_idx"] == media_id]
        
        # 결과 정리
        result = {
            "media_id": int(media_id),
            "basic_stats": {
                "total_clicks": int(media_info["total_clicks"]),
                "total_conversions": int(media_info["total_conversions"]),
                "global_cvr": round(float(media_info["mda_cvr_global"]), 4),
                "popularity_score": round(float(media_info["mda_pop_norm"]), 4)
            },
            "category_performance": []
        }
        
        # 카테고리별 성과 추가
        for _, cat_row in cat_performance.iterrows():
            result["category_performance"].append({
                "category": cat_row["ads_category"],
                "clicks": int(cat_row["cat_clicks"]),
                "conversions": int(cat_row["cat_conversions"]),
                "cvr": round(float(cat_row["cat_cvr"]), 4)
            })
        
        # 추가 데이터가 있으면 시간별 트렌드 분석
        if ads_time_data is not None:
            time_trend = self._analyze_media_time_trend(media_id, ads_time_data)
            result["time_trend"] = time_trend
        
        return result
    
    def _analyze_media_time_trend(self, media_id: int, ads_time_data: pd.DataFrame) -> Dict[str, Any]:
        """매체의 시간별 성과 트렌드 분석"""
        media_data = ads_time_data[ads_time_data["mda_idx"] == media_id].copy()
        
        if media_data.empty:
            return {"error": "해당 매체의 시간별 데이터가 없습니다."}
        
        media_data["date"] = pd.to_datetime(media_data["rpt_time_date"])
        media_data["clicks"] = pd.to_numeric(media_data.get("rpt_time_clk", 0), errors="coerce").fillna(0)
        media_data["conversions"] = pd.to_numeric(media_data.get("rpt_time_turn", 0), errors="coerce").fillna(0)
        
        # 일별 집계
        daily_stats = (media_data.groupby("date")
                      .agg(
                          daily_clicks=("clicks", "sum"),
                          daily_conversions=("conversions", "sum")
                      ).reset_index())
        
        daily_stats["daily_cvr"] = (
            daily_stats["daily_conversions"] / 
            daily_stats["daily_clicks"].replace(0, np.nan)
        ).fillna(0)
        
        # 트렌드 통계
        return {
            "total_days": len(daily_stats),
            "avg_daily_clicks": round(daily_stats["daily_clicks"].mean(), 2),
            "avg_daily_conversions": round(daily_stats["daily_conversions"].mean(), 2),
            "avg_daily_cvr": round(daily_stats["daily_cvr"].mean(), 4),
            "best_day": {
                "date": str(daily_stats.loc[daily_stats["daily_conversions"].idxmax(), "date"].date()),
                "conversions": int(daily_stats["daily_conversions"].max())
            },
            "recent_7days_avg_cvr": round(daily_stats.tail(7)["daily_cvr"].mean(), 4) if len(daily_stats) >= 7 else None
        }
    
    def get_top_media_by_conversions(self, top_n: int = 10) -> pd.DataFrame:
        """전환수 기준 상위 N개 매체사 조회"""
        if self.context_data is None:
            return pd.DataFrame()
        
        media_stats = self.context_data["media_stats"]
        
        top_media = (media_stats.sort_values("total_conversions", ascending=False)
                    .head(top_n)[["mda_idx", "total_clicks", "total_conversions", "mda_cvr_global"]]
                    .copy())
        
        top_media.columns = ["매체ID", "총클릭수", "총전환수", "전환율"]
        top_media["전환율"] = top_media["전환율"].round(4)
        top_media = top_media.reset_index(drop=True)
        
        return top_media
    
    def get_media_recommendations_for_category(self, category: str, top_n: int = 5) -> pd.DataFrame:
        """특정 카테고리에 최적화된 매체사 추천"""
        if self.context_data is None:
            return pd.DataFrame()
        
        category_stats = self.context_data["category_media_stats"]
        cat_media = category_stats[category_stats["ads_category"] == category]
        
        if cat_media.empty:
            return pd.DataFrame({"message": [f"'{category}' 카테고리 데이터가 없습니다."]})
        
        top_cat_media = (cat_media.sort_values("cat_cvr", ascending=False)
                        .head(top_n)[["mda_idx", "cat_clicks", "cat_conversions", "cat_cvr"]]
                        .copy())
        
        top_cat_media.columns = ["매체ID", f"{category}_클릭수", f"{category}_전환수", f"{category}_전환율"]
        top_cat_media[f"{category}_전환율"] = top_cat_media[f"{category}_전환율"].round(4)
        top_cat_media = top_cat_media.reset_index(drop=True)
        
        return top_cat_media


def evaluate_model(predictions: Dict[int, List[Tuple[int, float]]], 
                  ground_truth: Dict[int, List[int]], k: int = 10) -> Dict[str, float]:
    """
    모델 성능 평가
    """
    metrics = {"precision": [], "recall": [], "hit_rate": []}
    
    for ad_id in predictions:
        if ad_id not in ground_truth:
            continue
        
        pred_media = [media_id for media_id, _ in predictions[ad_id][:k]]
        true_media = set(ground_truth[ad_id])
        
        if len(true_media) == 0:
            continue
        
        # Precision@K
        precision = len(set(pred_media) & true_media) / len(pred_media)
        metrics["precision"].append(precision)
        
        # Recall@K
        recall = len(set(pred_media) & true_media) / len(true_media)
        metrics["recall"].append(recall)
        
        # Hit Rate@K
        hit_rate = 1.0 if len(set(pred_media) & true_media) > 0 else 0.0
        metrics["hit_rate"].append(hit_rate)
    
    return {
        f"precision@{k}": np.mean(metrics["precision"]),
        f"recall@{k}": np.mean(metrics["recall"]),
        f"hit_rate@{k}": np.mean(metrics["hit_rate"]),
        "evaluated_ads": len(metrics["precision"])
    }


# 사용 예시
if __name__ == "__main__":
    # 더미 데이터 생성 (실제 사용시에는 실제 데이터로 대체)
    def create_dummy_data():
        np.random.seed(42)
        
        # ads_time 더미 데이터 (실제 데이터 날짜 범위로 수정)
        dates = pd.date_range(start="2025-07-27", end="2025-08-29", freq="D")
        ads_time = []
        
        for date in dates:
            for ads_id in range(1, 101):  # 100개 광고
                for media_id in range(1, 21):  # 20개 매체
                    if np.random.random() < 0.3:  # 30% 확률로 노출
                        ads_time.append({
                            "rpt_time_date": date,
                            "ads_idx": ads_id,
                            "mda_idx": media_id,
                            "rpt_time_clk": np.random.poisson(10),
                            "rpt_time_turn": np.random.poisson(2)
                        })
        
        ads_time = pd.DataFrame(ads_time)
        
        # ads_list 더미 데이터
        categories = ["패션", "IT", "식품", "뷰티", "게임"]
        ads_list = pd.DataFrame({
            "ads_idx": range(1, 101),
            "ads_category": np.random.choice(categories, 100),
            "ads_reward_price": np.random.uniform(1000, 50000, 100),
            "ads_contract_price": np.random.uniform(800, 40000, 100)
        })
        
        return ads_time, ads_list
    
    print("더미 데이터 생성 중...")
    ads_time, ads_list = create_dummy_data()
    
    print("추천 시스템 초기화...")
    recommender = AdMediaRecommendationSystem(target_days=7)
    
    print("시간 윈도우 분할...")
    obs_data, target_data, window_info = recommender.split_time_windows(ads_time)
    print(f"관측 구간: {window_info['obs_start'].date()} ~ {window_info['obs_end'].date()}")
    print(f"타겟 구간: {window_info['target_start'].date()} ~ {window_info['target_end'].date()}")
    print(f"관측 데이터: {len(obs_data):,}건, 타겟 데이터: {len(target_data):,}건")
    
    print("학습 데이터 준비...")
    training_data = recommender.prepare_training_data(obs_data, target_data, ads_list, max_train_ads=50)
    print(f"학습 데이터 크기: {len(training_data)}")
    
    print("네거티브 샘플링...")
    training_data = recommender.downsample_negatives(training_data, neg_ratio=3)
    print(f"샘플링 후 크기: {len(training_data)}")
    
    print("모델 학습 중...")
    recommender.train_model(training_data)
    
    print("추천 생성...")
    predictions = recommender.predict_top_media(ads_list.head(10), top_k=5)
    
    print("\n=== 추천 결과 예시 ===")
    for ad_id, media_scores in list(predictions.items())[:3]:
        print(f"\n광고 ID {ad_id}의 상위 5개 매체:")
        for i, (media_id, score) in enumerate(media_scores, 1):
            print(f"  {i}. 매체 ID {media_id} (점수: {score:.4f})")
        
        # 예측 전환수 계산
        conversion_predictions = recommender.calculate_predicted_conversions(
            ad_id, media_scores, budget=1000000
        )
        
        print(f"  예측 성과 (예산 100만원):")
        for media_id, metrics in conversion_predictions.items():
            print(f"    매체 {media_id}: 예측 전환 {metrics['predicted_conversions']}개, "
                  f"ROI {metrics['predicted_roi']}%")
    
    print("\n=== 광고별 매체사 추천 및 예측 전환수 ===")
    
    # 1. 특정 광고들에 대한 매체사 추천
    test_ads = ads_list.head(5)  # 상위 5개 광고 테스트
    
    print("🎯 광고별 TOP-5 매체사 추천 결과:")
    print("="*80)
    
    for idx, ad_row in test_ads.iterrows():
        ad_id = int(ad_row["ads_idx"])
        ad_category = ad_row.get("ads_category", "미분류")
        ad_budget = ad_row.get("ads_reward_price", 1000000)  # 기본 예산 100만원
        
        print(f"\n📱 광고 ID: {ad_id} | 카테고리: {ad_category} | 예산: {ad_budget:,.0f}원")
        print("-" * 70)
        
        # 해당 광고의 매체사 추천 가져오기
        if ad_id in predictions:
            media_recommendations = predictions[ad_id][:5]  # TOP 5
            
            print("🏆 추천 매체사 순위:")
            for rank, (media_id, score) in enumerate(media_recommendations, 1):
                print(f"   {rank}. 매체 ID {media_id:2d} | 추천점수: {score:.4f}")
            
            # 예측 전환수 계산
            conversion_predictions = recommender.calculate_predicted_conversions(
                ad_id, media_recommendations, budget=float(ad_budget)
            )
            
            print(f"\n💰 예측 성과 (예산: {ad_budget:,.0f}원):")
            print(f"{'순위':<4}{'매체ID':<8}{'예측노출':<12}{'예측클릭':<12}{'예측전환':<12}{'예측ROI':<12}")
            print("-" * 60)
            
            for rank, (media_id, score) in enumerate(media_recommendations, 1):
                if media_id in conversion_predictions:
                    pred = conversion_predictions[media_id]
                    print(f"{rank:<4}{media_id:<8}"
                          f"{pred['predicted_impressions']:,<12}"
                          f"{pred['predicted_clicks']:,<12}"
                          f"{pred['predicted_conversions']:,<12}"
                          f"{pred['predicted_roi']:.1f}%")
            
            # 최고 성과 예측 매체
            if conversion_predictions:
                best_media = max(conversion_predictions.items(), 
                               key=lambda x: x[1]['predicted_conversions'])
                best_id, best_pred = best_media
                print(f"\n⭐ 최고 예측 성과: 매체 ID {best_id} → {best_pred['predicted_conversions']}전환 "
                      f"(ROI {best_pred['predicted_roi']:.1f}%)")
        else:
            print("   ❌ 해당 광고에 대한 추천 결과가 없습니다.")
    
    print("\n" + "="*80)
    
    # 2. 새로운 광고 시뮬레이션 (콜드스타트 테스트)
    print("\n🆕 신규 광고 시뮬레이션 (콜드스타트 테스트):")
    print("="*80)
    
    # 가상의 새 광고 생성
    new_ads = [
        {"ads_idx": 9001, "ads_category": "패션", "ads_reward_price": 2000000, "ads_contract_price": 1800000},
        {"ads_idx": 9002, "ads_category": "IT", "ads_reward_price": 5000000, "ads_contract_price": 4500000},
        {"ads_idx": 9003, "ads_category": "식품", "ads_reward_price": 1500000, "ads_contract_price": 1300000}
    ]
    
    new_ads_df = pd.DataFrame(new_ads)
    
    # 신규 광고들에 대한 매체사 추천
    new_predictions = recommender.predict_top_media(new_ads_df, top_k=5)
    
    for new_ad in new_ads:
        ad_id = new_ad["ads_idx"]
        category = new_ad["ads_category"] 
        budget = new_ad["ads_reward_price"]
        
        print(f"\n🆕 신규 광고 ID: {ad_id} | 카테고리: {category} | 예산: {budget:,.0f}원")
        print("-" * 70)
        
        if ad_id in new_predictions:
            media_recs = new_predictions[ad_id][:5]
            
            print("🎯 추천 매체사:")
            for rank, (media_id, score) in enumerate(media_recs, 1):
                print(f"   {rank}. 매체 ID {media_id} (점수: {score:.4f})")
            
            # 신규 광고 예측 전환수
            new_conv_pred = recommender.calculate_predicted_conversions(
                ad_id, media_recs, budget=budget
            )
            
            if new_conv_pred:
                total_predicted_conv = sum(p['predicted_conversions'] for p in new_conv_pred.values())
                avg_roi = sum(p['predicted_roi'] for p in new_conv_pred.values()) / len(new_conv_pred)
                
                print(f"   📊 전체 예측 전환수: {total_predicted_conv}개")
                print(f"   📈 평균 예측 ROI: {avg_roi:.1f}%")
                
                best_new = max(new_conv_pred.items(), key=lambda x: x[1]['predicted_conversions'])
                print(f"   ⭐ 최적 매체: ID {best_new[0]} → {best_new[1]['predicted_conversions']}전환")
    
    # 3. 광고별 매체 추천 요약 함수
    def get_ad_recommendations_summary(ad_id: int, top_k: int = 5, budget: float = 1000000):
        """특정 광고의 매체 추천 요약"""
        print(f"\n📊 광고 ID {ad_id} 추천 요약 (예산: {budget:,.0f}원)")
        print("="*50)
        
        # 광고 정보 
        ad_info = ads_list[ads_list["ads_idx"] == ad_id]
        if not ad_info.empty:
            category = ad_info.iloc[0].get("ads_category", "미분류")
            print(f"카테고리: {category}")
        
        # 단일 광고 DataFrame으로 예측
        single_ad_df = pd.DataFrame([{"ads_idx": ad_id, "ads_category": category, 
                                     "ads_reward_price": budget, "ads_contract_price": budget*0.9}])
        
        single_pred = recommender.predict_top_media(single_ad_df, top_k=top_k)
        
        if ad_id in single_pred:
            recommendations = single_pred[ad_id]
            conv_pred = recommender.calculate_predicted_conversions(ad_id, recommendations, budget)
            
            print(f"TOP-{top_k} 추천 매체:")
            for i, (mid, score) in enumerate(recommendations, 1):
                conv_info = conv_pred.get(mid, {})
                conv_num = conv_info.get('predicted_conversions', 0)
                roi = conv_info.get('predicted_roi', 0)
                print(f"  {i}. 매체 {mid}: {conv_num}전환 예상 (ROI {roi:.1f}%)")
            
            return {"recommendations": recommendations, "predictions": conv_pred}
        else:
            print("추천 결과 없음")
            return None
    
    # 4. 사용 예시
    print(f"\n🔧 광고별 매체 추천 함수 사용 예시:")
    print("="*80)
    
    # 특정 광고들에 대한 개별 추천
    for test_ad_id in [1, 5, 10]:
        result = get_ad_recommendations_summary(test_ad_id, top_k=3, budget=1500000)
    
    print(f"\n" + "="*80)
    print("✅ 광고별 매체사 추천 시스템 완료!")
    print("💡 주요 기능:")
    print("   • 각 광고마다 최적 매체사 TOP-N 추천")
    print("   • 예산별 예측 전환수 계산") 
    print("   • 신규 광고 콜드스타트 대응")
    print("   • ROI 기반 매체 성과 예측")
    print("="*80)

더미 데이터 생성 중...
추천 시스템 초기화...
시간 윈도우 분할...
📅 실제 데이터 범위: 2025-07-27 ~ 2025-08-29
관측 구간: 2025-07-27 ~ 2025-08-22
타겟 구간: 2025-08-23 ~ 2025-08-29
관측 데이터: 16,013건, 타겟 데이터: 4,161건
학습 데이터 준비...
학습 데이터 크기: 1000
네거티브 샘플링...
샘플링 후 크기: 1000
모델 학습 중...
모델 학습 완료 - 사용된 피처: ['mda_cvr_global', 'mda_pop_norm', 'cat_cvr', 'price_reward_log', 'price_ratio']
추천 생성...

=== 추천 결과 예시 ===

광고 ID 1의 상위 5개 매체:
  1. 매체 ID 18 (점수: 0.9647)
  2. 매체 ID 7 (점수: 0.7979)
  3. 매체 ID 14 (점수: -0.2487)
  4. 매체 ID 11 (점수: -0.5345)
  5. 매체 ID 13 (점수: -0.9942)
  예측 성과 (예산 100만원):
    매체 18: 예측 전환 3646개, ROI 18234.16%
    매체 7: 예측 전환 3262개, ROI 16313.29%
    매체 14: 예측 전환 -835개, ROI -4179.51%
    매체 11: 예측 전환 -2092개, ROI -10461.03%
    매체 13: 예측 전환 -3925개, ROI -19627.66%

광고 ID 2의 상위 5개 매체:
  1. 매체 ID 6 (점수: 2.2477)
  2. 매체 ID 8 (점수: 0.6910)
  3. 매체 ID 4 (점수: 0.5352)
  4. 매체 ID 7 (점수: 0.3199)
  5. 매체 ID 5 (점수: -0.1489)
  예측 성과 (예산 100만원):
    매체 6: 예측 전환 8992개, ROI 44964.12%
    매체 8: 예측 전환 2619개, ROI 13097.7%
    매체 4: 예측 전환 204